In [1]:
import pandas as pd
import numpy as np
import dask as ds

In [16]:
data = pd.read_table("../data/drugs.tsv", sep='\t')
data = data.set_index('CASEID')
data

,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,48694667,1,99,99,19,2012,7,1,1,99,...,1,4,1,1,2,2,2,4398.40,30017,1
2,88530883,1,99,99,14,9999,99,2,93,93,...,1,4,1,1,1,1,2,1419.19,30052,2
3,33251077,1,99,99,14,9999,99,1,2,99,...,1,99,9,9,3,3,2,14052.62,30028,1
4,37814127,1,99,99,16,9999,99,4,93,93,...,1,4,1,1,1,1,2,10848.18,30055,2
5,18762590,1,99,99,14,9999,99,4,93,93,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55156,13675473,2,99,99,991,9991,91,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1
55157,49609908,1,99,99,16,9999,99,4,93,93,...,1,1,1,1,2,2,2,2296.28,30057,2
55158,81795924,2,99,99,991,9991,91,91,91,91,...,1,4,1,1,3,3,2,17180.64,30026,1


In [17]:
# we only want ages 12 - 25
data = data[(data['CATAGE'] == 1) | (data['CATAGE'] == 2)]
data

,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,48694667,1,99,99,19,2012,7,1,1,99,...,1,4,1,1,2,2,2,4398.40,30017,1
2,88530883,1,99,99,14,9999,99,2,93,93,...,1,4,1,1,1,1,2,1419.19,30052,2
3,33251077,1,99,99,14,9999,99,1,2,99,...,1,99,9,9,3,3,2,14052.62,30028,1
8,35106150,1,99,99,22,9999,99,2,93,93,...,1,1,1,1,2,2,2,3211.24,30024,1
9,67182690,1,99,99,16,9999,99,1,20,99,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55153,31776420,2,99,99,991,9991,91,91,91,91,...,1,4,1,1,2,2,2,1649.35,30024,2
55155,10772244,1,99,99,18,9999,99,3,93,93,...,1,1,1,1,1,1,2,4594.84,30002,2
55156,13675473,2,99,99,991,9991,91,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1


In [18]:
data['QUESTID2'].value_counts().sum() # length is the same, each Questid is unique
data['CIGOFRSM'].value_counts() # we can see there is majority NaN

99    21177
4     12512
3      1998
2       166
1        13
97        6
94        5
98        1
Name: CIGOFRSM, dtype: int64

In [19]:
# drop columns where majority are NaN
data = data.loc[:, (data == 99).mean() <= .5]
data = data.loc[:, (data == 999).mean() <= .5]
data = data.loc[:, (data == 9999).mean() <= .5]
data

,QUESTID2,CIGEVER,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,CIG30AV,CIG30BR2,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,48694667,1,19,2012,7,1,1,99,2,9994,...,1,4,1,1,2,2,2,4398.40,30017,1
2,88530883,1,14,9999,99,2,93,93,93,9993,...,1,4,1,1,1,1,2,1419.19,30052,2
3,33251077,1,14,9999,99,1,2,99,1,112,...,1,99,9,9,3,3,2,14052.62,30028,1
8,35106150,1,22,9999,99,2,93,93,93,9993,...,1,1,1,1,2,2,2,3211.24,30024,1
9,67182690,1,16,9999,99,1,20,99,3,112,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55153,31776420,2,991,9991,91,91,91,91,91,9991,...,1,4,1,1,2,2,2,1649.35,30024,2
55155,10772244,1,18,9999,99,3,93,93,93,9993,...,1,1,1,1,1,1,2,4594.84,30002,2
55156,13675473,2,991,9991,91,91,91,91,91,9991,...,1,2,1,1,3,3,2,679.36,30003,1
